<div class="alert alert-info"; style="border-left: 7px solid blue">
<b>Student's comment</b>
    
**Data Analyst Course Final Project - Student Project**
    
**E-Commerce Product Range Analysis**

<div class="alert alert-info"; style="border-left: 7px solid blue">
<b>Student's comment</b>
    
**Step 1 - Task Decomposition**
    
First, we will need to examine our products: 
    
* We will assign and categorize items. 
    
* Find out the best/worst selling products, based on quantity and price.
    
* Go over the invoices and check if there is there an increase in sales during specific weeks/months.
    
* One way to increase sales is to offer product bundles or promotions/discounts. Is there any indication that we provide this?
    
* Does the appeareance of a low selling item in a bundle dissuade potential customers from buying them?
    
* Is there a special pricing contract for those that buy in bulk (i.e, businesses)?
    

Once we answer these questions, we would be able to build a typical purchase portfolio, and find out which factors determine purchases, and purchase sizes.

In [1]:
import pandas as pd
from scipy import stats as st
import numpy as np
import math as mth
from plotly import graph_objects as go

In [2]:
data = pd.read_csv('/datasets/ecommerce_dataset_us.csv', sep='\t')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,11/29/2018 08:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,11/29/2018 08:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,11/29/2018 08:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,11/29/2018 08:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,11/29/2018 08:26,3.39,17850.0
...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/07/2019 12:50,0.85,12680.0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/07/2019 12:50,2.10,12680.0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/07/2019 12:50,4.15,12680.0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/07/2019 12:50,4.15,12680.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 28.9+ MB


<div class="alert alert-info"; style="border-left: 7px solid blue">
<b>Student's comment</b>
    
We need to convert the InvoiceDate column to a timestamp. After that, our data is set and we can check for duplicates and missing values.

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 28.9+ MB


In [5]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
dtype: int64

<div class="alert alert-info"; style="border-left: 7px solid blue">
<b>Student's comment</b>
    
For the description column, let's check if the missing description is simply an error (meaning, the item exists and has a  StockCode, but has no description). For that, we will have to create a dataframe with the missing values, and use a for loop to iterate over the unique code values, and fill them accordingly. 
    
We will run this same code to try and fill missing customerID numbers based on the invoice number.

In [6]:
missing_values = data['Description'].isnull()
missing_rows = data[missing_values]
stock_codes = missing_rows['StockCode'].unique()

for stock_code in stock_codes:
    non_null_descriptions = data.loc[data['StockCode'] == stock_code, 'Description'].dropna()
    
    if len(non_null_descriptions) > 0:
        description = non_null_descriptions.iloc[0]
        
        data.loc[(data['StockCode'] == stock_code) & missing_values, 'Description'] = description

In [7]:
missing_values_customer_id = data['CustomerID'].isnull()
missing_rows_customer_id = data[missing_values_customer_id]
invoice_numbers = missing_rows_customer_id['InvoiceNo'].unique()

for invoice_number in invoice_numbers:
    non_null_customer_ids = data.loc[data['InvoiceNo'] == invoice_number, 'CustomerID'].dropna()
    
    if len(non_null_customer_ids) > 0:
        customer_id = non_null_customer_ids.iloc[0]
        
        data.loc[(data['InvoiceNo'] == invoice_number) & missing_values_customer_id, 'CustomerID'] = customer_id